In [88]:
from __future__ import division
from psychopy.visual import ImageStim, TextStim, Window
from psychopy import core, event, gui, data, logging
import numpy as np
import pandas as pd
import os

from routines import Routine

# Code for the (simplified) reinforcement learning task from Fontanesi and colleagues (https://doi.org/10.3758/s13423-018-1554-2)

#general settings
expName = 'Group A'
screen_size = [2000, 1200]
frames_per_second = 60
full_screen = False
background_color = '#eeeeee'
n_trials = 30

# trial settings
choice_keys = ['q', 'p']
escape_key = 'escape'
choice_time_limit = 5
feedback_duration = 2
fixation_duration = 1

#stimuli settings
text_color = 'black'
text_height = 50
options_x_offset = 200
image_size = 100

#store info about the experiment session
dlg = gui.Dlg(title=expName)
dlg.addField('Participant:', 1)
dlg.addField('Age:', 25)
dlg.addField('Gender:', choices=['female', 'male', 'prefer not to disclose'])
dlg.addField("condition", choices = ["A", "B"]) #change into A and B
dlg.addField('Handedness:', choices=['right', 'left', 'both'])
dlg.show()

expInfo = dict(zip(['participant', 'age', 'gender', 'hand'], dlg.data))
expInfo['date'] = data.getDateStr()  # add a simple timestamp
expInfo['expName'] = expName # add the experiment name
if dlg.OK:  # then the user pressed OK
    print(expInfo)
else:
    print(expInfo)
    core.quit()

#check if data folder exists
directory=os.path.join(os.getcwd(), 'data')
if not os.path.exists(directory):
    os.makedirs(directory)

#create file name for storing data
fileName = os.path.join('data', '%s_%s_%s' % (expName, expInfo['participant'], expInfo['date']))

#save a log file
logFile = logging.LogFile(fileName + '.log', level=logging.DEBUG)
logging.console.setLevel(logging.WARNING)  # this outputs to the screen, not a file

#create a window
mywin = Window(screen_size, units='pix', color=background_color, fullscr=full_screen)

#create some stimuli
left_feedback = TextStim(win=mywin, color=text_color, pos=(-options_x_offset, 0), height=text_height)
right_feedback = TextStim(win=mywin, color=text_color, pos=(options_x_offset, 0), height=text_height)

left_stimuli = TextStim(win=mywin, pos=(-options_x_offset, 0), height=text_height)
right_stimuli = TextStim(win=mywin, pos=(options_x_offset, 0), height=text_height)

#Lotterien (2x2 outcomes, ähnlichem EV, verschiedene Varianz)
#Zufällige Zahlen 1 bis 10 x 30
a = np.random.choice(a = np.arange(1,11), size = 40, replace = True)

#Zufällige Wahrscheinlichkeiten x 30 (px)
b = np.random.choice(a = np.arange(0.3, 0.7, 0.05), size = 40, replace = True)

#EVx: x * px
c = a * b

# Y = evx/py
d = c / (1 - b)

#py: 1 - px
e = 1 - b

#Evy: y * py
f = d * e

#creating data frame for nornal Lotteries
x = {"x": a, "px": b, "evx": c, "y": d, "py": e, "evy": f}
normallot = pd.DataFrame(x)

#Lotterien (plus 1000 bei outcomes) #within subject
# Lotterie mit grossen Outcomes
# x + 1000
abig = a + 1000
# px (gleich wie bei normal lotteries)
bbig = b
# evx = abig * bbig
cbig = abig * bbig
# Y = evx/py
dbig = cbig / (1 - bbig)
# py = 1 - px
ebig = 1 - bbig
# evy = y * py
fbig = dbig * ebig
#creating data frame
x = {"x": abig, "px": bbig, "evx": cbig, "y": dbig, "py": ebig, "evy": fbig}
biglot = pd.DataFrame(x)

#Lotterien mit rare event 
#Zufällige Zahlen 1 bis 10 x 30
ahard = np.random.choice(a = np.arange(1,11), size = 40, replace = True)

#Zufällige Wahrscheinlichkeiten x 30 (px zwieschen 0.1 und 0.3)
bhard = np.random.choice(a = np.arange(0.1, 0.3, 0.05), size = 40, replace = True)

#EVx: x * px
chard = ahard * bhard

# Y = evx/py
dhard = chard / (1 - bhard)

#py: 1 - px
ehard = 1 - bhard

#Evy: y * py
fhard = dhard * ehard

#creating data frame for nornal Lotteries
x = {"x": ahard, "px": bhard, "evx": chard, "y": dhard, "py": ehard, "evy": fhard}
hardlot = pd.DataFrame(x)
hardlot

#Lotterien mit hohen outcomes und rare events
#Zufällige Zahlen 1 bis 10 x 30
abigh = abig

#Zufällige Wahrscheinlichkeiten x 30 (px zwieschen 0.1 und 0.3)
bbigh = b

#EVx: x * px
cbigh = abigh * bbigh

# Y = evx/py
dbigh = cbigh / (1 - bbigh)

#py: 1 - px
ebigh = 1 - bbigh

#Evy: y * py
fbigh = dbigh * ebigh

#creating data frame for nornal Lotteries
x = {"x": abigh, "px": bbigh, "evx": cbigh, "y": dbigh, "py": ebigh, "evy": fbigh}
bighlot = pd.DataFrame(x)

#fixation cross
fixation_cross = TextStim(win=mywin, text='+', color=text_color)

#create the dataframe
data = pd.DataFrame([])

#draw the stimuli
trial_routine = Routine(window=mywin, frames_per_second=frames_per_second, escape_key=escape_key)


for t in range(n_trials):
    # put here things that change every trial$
    if "condition" == "A": #A = easy option/condition
        stimuli = {'left_stimuli': [normallot.loc[t, ["x", "px", "y", "py"]] , biglot.loc[t, ["x", "px", "y", "py"]]],
         'right_stimuli': [biglot.loc[t, ["x", "px", "y", "py"]], normallot.loc[t, ["x", "px", "y", "py"]]],
         'left_feedback': [np.random.choice(a = normallot.loc[t, ["x", "y"]],  p= normallot.loc[t, ["px", "py"]]), 
                            np.random.choice(a = biglot.loc[t, ["x", "y"]],  p= biglot.loc[t, ["px", "py"]])], 
         'right_feedback': [np.random.choice(a = biglot.loc[t, ["x", "y"]],  p= biglot.loc[t, ["px", "py"]]), 
                            np.random.choice(a = normallot.loc[t, ["x", "y"]],  p= normallot.loc[t, ["px", "py"]])]
        }
    else:
         stimuli = {'left_stimuli': [hardlot.loc[t, ["x", "px", "y", "py"]] , bighlot.loc[t, ["x", "px", "y", "py"]]],
         'right_stimuli': [bighlot.loc[t, ["x", "px", "y", "py"]], hardlot.loc[t, ["x", "px", "y", "py"]]],
         'left_feedback': [np.random.choice(a = hardlot.loc[t, ["x", "y"]],  p= hardlot.loc[t, ["px", "py"]]), 
                            np.random.choice(a = bighlot.loc[t, ["x", "y"]],  p= bighlot.loc[t, ["px", "py"]])], 
         'right_feedback': [np.random.choice(a = bighlot.loc[t, ["x", "y"]],  p= bighlot.loc[t, ["px", "py"]]), 
                            np.random.choice(a = hardlot.loc[t, ["x", "y"]],  p= hardlot.loc[t, ["px", "py"]])]
        }
    
    randomizer = np.random.choice(a = [0,1])   
    if randomizer == 0:
        left_stimuli = list(stimuli.values["left_stimuli"])[0][0:4]
        left_feedback = list(stimuli["left_feedback"])
        right_stimuli = list(stimuli.values["right_stimuli"])[0][0:4]
        right_feedback = list(stimuli["right_feedback"])
        left_stimuli.text = "%s" % left_stimuli
        left_feedback.text = '%s' % np.random.choice(a = left_feedback, p = left_stimuli["px", "py"]
        right_stimuli.text = "%s" % right_stimuli
        right_feedback.text = '%s' % np.random.choice(a = right_feedback, p = right_stimuli["px", "py"]
    else:
        left_stimuli = list(stimuli.values["left_stimuli"])[1][0:4]
        left_feedback = list(stimuli["left_feedback"])
        right_stimuli = list(stimuli.values["right_stimuli"])[1][0:4]
        right_feedback = list(stimuli["right_feedback"])                                              
        left_stimuli.text = "%s" % left_stimuli
        left_feedback.text = '%s' % np.random.choice(a = left_feedback, p = left_stimuli["px", "py"]
        right_stimuli.text = "%s" % right_stimuli
        right_feedback.text = '%s' % np.random.choice(a = right_feedback, p = right_stimuli["px", "py"]
    
    # first event
    trial_routine.wait_for_time_limit(
        components=[fixation_cross], 
        time_seconds=fixation_duration, 
        label='fixation_cross')
    
    # second event
    #antwort registrieren und speichern
    key, rt = trial_routine.wait_for_keys(
        components=[left_stimuli, right_stimuli],
        valid_keys=choice_keys,
        label='gamble_choice')
    
    data = data.append({'trial': t, "choice": key, "f_left": left_feedback, "f_right" : right_feedback
    }, ignore_index=True)
    
    # third event 
    trial_routine.wait_for_time_limit(
        components=[left_feedback, right_feedback], 
        time_seconds=feedback_duration, 
        label= "feedback")
    
    
       
    #save data to file
    for label in expInfo.keys():
        data[label] = expInfo[label]
    data.to_csv(fileName + '.csv')
    
#cleanup
mywin.close()
core.quit()

SyntaxError: invalid syntax (<ipython-input-88-db484a5b71b7>, line 198)

In [85]:
right_stimuli = list(stimuli["right_stimuli"])[0][0:4]
right_stimuli.values.round

array([1.002e+03, 5.000e-01, 1.002e+03, 5.000e-01])

In [22]:
    if "condition" == "A": #A = easy option/condition
        stimuli = {'left_stimuli': [normallot.loc[t, ["x", "px", "y", "py"]] , biglot.loc[t, ["x", "px", "y", "py"]]],
         'right_stimuli': [biglot.loc[t, ["x", "px", "y", "py"]], normallot.loc[t, ["x", "px", "y", "py"]]],
         'left_feedback': [np.random.choice(a = normallot.loc[t, ["x", "y"]],  p= normallot.loc[t, ["px", "py"]]), 
                            np.random.choice(a = biglot.loc[t, ["x", "y"]],  p= biglot.loc[t, ["px", "py"]])], 
         'right_feedback': [np.random.choice(a = biglot.loc[t, ["x", "y"]],  p= biglot.loc[t, ["px", "py"]]), 
                            np.random.choice(a = normallot.loc[t, ["x", "y"]],  p= normallot.loc[t, ["px", "py"]])]
        }
    else:
         stimuli = {'left_stimuli': [hardlot.loc[t, ["x", "px", "y", "py"]] , bighlot.loc[t, ["x", "px", "y", "py"]]],
         'right_stimuli': [bighlot.loc[t, ["x", "px", "y", "py"]], hardlot.loc[t, ["x", "px", "y", "py"]]],
         'left_feedback': [np.random.choice(a = hardlot.loc[t, ["x", "y"]],  p= hardlot.loc[t, ["px", "py"]]), 
                            np.random.choice(a = bighlot.loc[t, ["x", "y"]],  p= bighlot.loc[t, ["px", "py"]])], 
         'right_feedback': [np.random.choice(a = bighlot.loc[t, ["x", "y"]],  p= bighlot.loc[t, ["px", "py"]]), 
                            np.random.choice(a = hardlot.loc[t, ["x", "y"]],  p= hardlot.loc[t, ["px", "py"]])]
        }
    asd = list(stimuli["right_feedback"])[1]
    asd

2.0000000000000004

In [7]:
#Lotteriensammlung
import numpy as np
import pandas as pd
#Zufällige Zahlen 1 bis 10 x 30
a = np.random.choice(a = np.arange(1,11), size = 30, replace = True)

#Zufällige Wahrscheinlichkeiten x 30 (px)
b = np.random.choice(a = np.arange(0.3, 0.7, 0.05), size = 30, replace = True)

#EVx: x * px
c = a * b

# Y = evx/py
d = c / (1 - b)

#py: 1 - px
e = 1 - b

#Evy: y * py
f = d * e

#varx
g = {"x": a, "px": b}
h = pd.DataFrame(g)
i = np.var(h, axis = 1)

#vary
j = {"y": d, "py": e}
k = pd.DataFrame(j)
l = np.var(k, axis = 1)

#creating data frame for nornal Lotteries
x = {"x": a, "px": b, "evx": c, "varx": i, "y": d, "py": e, "evy": f, "vary": l}
normallot = pd.DataFrame(x)
normallot

,x,px,evx,varx,y,py,evy,vary
0,10,0.45,4.50,22.800625,8.181818,0.55,4.50,14.561162
1,6,0.65,3.90,7.155625,11.142857,0.35,3.90,29.121441
2,10,0.60,6.00,22.090000,15.000000,0.40,6.00,53.290000
3,6,0.60,3.60,7.290000,9.000000,0.40,3.60,18.490000
4,10,0.50,5.00,22.562500,10.000000,0.50,5.00,22.562500
5,9,0.60,5.40,17.640000,13.500000,0.40,5.40,42.902500
6,5,0.45,2.25,5.175625,4.090909,0.55,2.25,3.134509
7,6,0.50,3.00,7.562500,6.000000,0.50,3.00,7.562500
8,2,0.65,1.30,0.455625,3.714286,0.35,1.30,2.829605
9,1,0.65,0.65,0.030625,1.857143,0.35,0.65,0.567870


In [8]:
# Lotterie mit grossen Outcomes
# x + 1000
abig = np.random.choice(a = np.arange(1,11), size = 30, replace = True) + 1000
# px (gleich wie bei normal lotteries)
bbig = b
# evx = abig * bbig
cbig = abig * bbig
# Y = evx/py
dbig = cbig / (1 - bbig)
# py = 1 - px
ebig = 1 - bbig
# evy = y * py
fbig = dbig * ebig
#varx
gbig = {"x": abig, "px": b}
hbig = pd.DataFrame(gbig)
ibig = np.var(hbig, axis = 1)

#vary
jbig = {"y": dbig, "py": ebig}
kbig = pd.DataFrame(jbig)
lbig = np.var(kbig, axis = 1)
#creating data frame
x = {"x": abig, "px": bbig, "evx": cbig, "varx": ibig, "y": dbig, "py": ebig, "evy": fbig, "vary": lbig}
biglot = pd.DataFrame(x)
biglot

,x,px,evx,varx,y,py,evy,vary
0,1010,0.45,454.50,254797.800625,826.363636,0.55,454.50,170492.040501
1,1001,0.65,650.65,250175.030625,1859.000000,0.35,650.65,863644.955625
2,1006,0.60,603.60,252707.290000,1509.000000,0.40,603.60,568968.490000
3,1010,0.60,606.00,254722.090000,1515.000000,0.40,606.00,573503.290000
4,1010,0.50,505.00,254772.562500,1010.000000,0.50,505.00,254772.562500
5,1003,0.60,601.80,251201.440000,1504.500000,0.40,601.80,565579.202500
6,1005,0.45,452.25,252280.175625,822.272727,0.55,452.25,168807.060129
7,1009,0.50,504.50,254268.062500,1009.000000,0.50,504.50,254268.062500
8,1001,0.65,650.65,250175.030625,1859.000000,0.35,650.65,863644.955625
9,1009,0.65,655.85,254192.430625,1873.857143,0.35,655.85,877507.253584


In [9]:
# schwere Lotterie
#Zufällige Zahlen 1 bis 10 x 30
ahard = np.random.choice(a = np.arange(1,11), size = 30, replace = True)

#Zufällige Wahrscheinlichkeiten x 30 (px zwieschen 0.1 und 0.3)
bhard = np.random.choice(a = np.arange(0.1, 0.3, 0.05), size = 30, replace = True)

#EVx: x * px
chard = ahard * bhard

# Y = evx/py
dhard = chard / (1 - bhard)

#py: 1 - px
ehard = 1 - bhard

#Evy: y * py
fhard = dhard * ehard

#varx
ghard = {"x": ahard, "px": bhard}
hhard = pd.DataFrame(ghard)
ihard = np.var(hhard, axis = 1)

#vary
jhard = {"y": dhard, "py": ehard}
khard = pd.DataFrame(jhard)
lhard = np.var(khard, axis = 1)

#creating data frame for nornal Lotteries
x = {"x": ahard, "px": bhard, "evx": chard, "varx": ihard, "y": dhard, "py": ehard, "evy": fhard, "vary": lhard}
hardlot = pd.DataFrame(x)
hardlot

,x,px,evx,varx,y,py,evy,vary
0,10,0.25,2.50,23.765625,3.333333,0.75,2.50,1.668403
1,10,0.25,2.50,23.765625,3.333333,0.75,2.50,1.668403
2,5,0.10,0.50,6.002500,0.555556,0.90,0.50,0.029660
3,10,0.15,1.50,24.255625,1.764706,0.85,1.50,0.209172
4,1,0.15,0.15,0.180625,0.176471,0.85,0.15,0.113410
5,6,0.25,1.50,8.265625,2.000000,0.75,1.50,0.390625
6,5,0.25,1.25,5.640625,1.666667,0.75,1.25,0.210069
7,4,0.20,0.80,3.610000,1.000000,0.80,0.80,0.010000
8,6,0.10,0.60,8.702500,0.666667,0.90,0.60,0.013611
9,4,0.15,0.60,3.705625,0.705882,0.85,0.60,0.005192


In [10]:
# schwere + big Lotterie
#Zufällige Zahlen 1 bis 10 x 30
abigh = abig

#Zufällige Wahrscheinlichkeiten x 30 (px zwieschen 0.1 und 0.3)
bbigh = b

#EVx: x * px
cbigh = abigh * bbigh

# Y = evx/py
dbigh = cbigh / (1 - bbigh)

#py: 1 - px
ebigh = 1 - bbigh

#Evy: y * py
fbigh = dbigh * ebigh

#varx
gbigh = {"x": abigh, "px": bbigh}
hbigh = pd.DataFrame(gbigh)
ibigh = np.var(hbigh, axis = 1)

#vary
jbigh = {"y": dbigh, "py": ebigh}
kbigh = pd.DataFrame(jbigh)
lbigh = np.var(kbigh, axis = 1)

#creating data frame for nornal Lotteries
x = {"x": abigh, "px": bbigh, "evx": cbigh, "varx": ibigh, "y": dbigh, "py": ebigh, "evy": fbigh, "vary": lbigh}
bighlot = pd.DataFrame(x)
bighlot

,x,px,evx,varx,y,py,evy,vary
0,1010,0.45,454.50,254797.800625,826.363636,0.55,454.50,170492.040501
1,1001,0.65,650.65,250175.030625,1859.000000,0.35,650.65,863644.955625
2,1006,0.60,603.60,252707.290000,1509.000000,0.40,603.60,568968.490000
3,1010,0.60,606.00,254722.090000,1515.000000,0.40,606.00,573503.290000
4,1010,0.50,505.00,254772.562500,1010.000000,0.50,505.00,254772.562500
5,1003,0.60,601.80,251201.440000,1504.500000,0.40,601.80,565579.202500
6,1005,0.45,452.25,252280.175625,822.272727,0.55,452.25,168807.060129
7,1009,0.50,504.50,254268.062500,1009.000000,0.50,504.50,254268.062500
8,1001,0.65,650.65,250175.030625,1859.000000,0.35,650.65,863644.955625
9,1009,0.65,655.85,254192.430625,1873.857143,0.35,655.85,877507.253584
